# Geothermal-based Thermal Mitigation System Model
Created by: Erik Janssen   
Date: September 2020

## Introduction
This Jupyter notebook is a companion to the paper titled "Thermal Mitigation of Stormwater Management Pond Outflows Using Geothermal Cooling" submitted to the Journal of Water Management Modeling. It provides a Python function that numerically solves the analytical geothermal system model. Different functions would be required depending on the variable being solved for. The function provided below solves for the pond outflow temperature downstream of the geothermal cooling system (T<sub>p2</sub> in the paper). 

In [22]:
# Import Libraries
import math #Contains library for natural logirthm function
from scipy import optimize as optimize #Contains library for numerical solver

# Define solver function
def predicted(T_g, T_p1, flow_p, flow_h, R_SHX, R_GHX, L_SHX, L_GHX, rho_p, rho_h, cap_p, cap_h):
    '''
    This function calculates a numerical solution for pond outflow downstream of the geothermal cooling system (T_p2)
    based on other system parameters that must be defined:    
    T_g - Ground Temperature in units C
    T_p1 - Pond outflow temperature upstream of the geothermal system in units C
    flow_p - Pond flowrate in units L/s
    flow_h - Hydronic flowrate in units L/min
    R_SHX - Thermal resistance of the SHX in units mC/W
    R_GHX - Thermal resistance of the GHX in units mC/W
    L_GHX - Borehole length (depth of borehole(s) multiplied by number of boreholes) (m)
    L_SHX - Total length of pipe in the SHX that is immersed in the pond outflow (m)
    rho_p - Density of water kg/m3
    rho_h - Density of the heat transfer fluid kg/m3
    cap_p - Specific heat capacity of water J/(kgC)
    cap_h - Specific heat capacity of water J/(kgC)    
    '''
    # Change flow units to m3/s
    flow_p = flow_p*(1/1000)
    flow_h = flow_h*(1/60)*(1/1000)

    # There is a point discontinuity of these terms are the same - so if they are the same make them slightly different
    if (rho_p * flow_p * cap_p) == (rho_h * flow_h * cap_h):
        flow_p = flow_p * 1.001

    # Function that defines the heat transfer as a function of the pond outlet temperature and other set parameters
    def q(T_p2):
        q_ = rho_p * flow_p * cap_p * (T_p1 - T_p2)
        return q_

    # Function that defines the hydronic system return temperature to the borehole as a function of the pond outlet 
    # temperature and other set parameters
    def T_h2(T_p2):
        T_h2_ = q(T_p2) * R_GHX / L_GHX + q(T_p2)/(2 * rho_h * flow_h * cap_h) + T_g
        return T_h2_

    # Function that defines the hydronic system supply temperature to the borehole as a function of the pond outlet 
    # temperature and other set parameters
    def T_h1(T_p2):
        T_h1_ = T_h2(T_p2) - q(T_p2) / (rho_h * flow_h * cap_h)
        if T_h1_ > T_g: #supply can't be cooler than ground itself...
            T_h1_x = T_h1_
        else:
            T_h1_x = T_g
        return T_h1_x

    # Function to be solved numerically for the pond outflow temperature (T_p2)
    def y(T_p2):
        ln_numer = T_p2 - T_h1(T_p2)
        ln_denom = T_p1 - T_h2(T_p2)
        rat_denom = T_p1 - T_h2(T_p2) - (T_p2 - T_h1(T_p2))
        if ln_numer/ln_denom > 0:
            if rat_denom < -0.00001: #prevents things from blowing up when numerically solving
                y_ =  q(T_p2) * R_SHX * math.log(ln_numer/ln_denom)/rat_denom + L_SHX
            else:
                y_ = -1 # Used as a flag to identify no solution
        else:
            y_ = -1 # Used as a flag to identify no solution
        return y_

    # This is the denominator of the natural logarithm expression in y; it will be used in this function to 
    # identify when the ln argument blows up to infinity
    def ln_den(T_p2):
        y = T_p1 - T_h2(T_p2)
        return y

    # This is when the argument of the natural logarithm blows up to infinity - need to avoid in the numerical solution
    def blows_up():
        blown_up = optimize.fsolve(ln_den, 15)
        return blown_up    

    # Need to provide an initial point for the numerical solver but there are vertical asymptotes that 
    # need to be avoided - this function is used to avoid them
    def post_blw_up_factor():
        if (flow_h < 0.0002):
            factor = 0.01
        else:
            factor = 0.0005
        return factor       

    # Define the initial geuss for the numerical solver
    post_blowup = blows_up()+post_blw_up_factor()

    # Solve the equations   
    Tp2_C = optimize.fsolve(y, post_blowup) # Pond outflow temperature downstream of the geothermal system in units C
    Th1_C = T_h1(Tp2_C) # Hydronic system supply temp to surface water heat exchanger in units C
    Th2_C = T_h2(Tp2_C) # Hydronic system return temp to borehole in units C
    q_ = q(Tp2_C) # Cooling power in units kW
    result = [Tp2_C[0],Th1_C[0],Th2_C[0],q_[0]]
    
    return result  

Here is the solver used in an example:

In [45]:
result = predicted(T_g=10, T_p1=30, flow_p=0.4, flow_h=17, R_SHX=0.14, R_GHX=0.21, L_SHX=230, L_GHX=183, rho_p=1000, 
          rho_h=1000, cap_p=4184, cap_h=4184)

In [46]:
print('For the provided parameters, the system provides ' + str(int(result[3])) + ' W of cooling and the outflow \
temperature downstream of the goethermal system is ' + str(round(result[0],1)))

For the provided parameters, the system provides 9693 W of cooling and the outflow temperature downstream of the goethermal system is 24.2
